In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a regression problem.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [50]:
# third-party libraries
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame, impute
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters

In [4]:
# source code
from src.utils import load_data, load_config
from src.data_preprocessing import calculate_RUL

In [5]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [6]:
np.random.seed(42)

# Paths

In [7]:
os.chdir("../") # set working directory to root of project
#os.getcwd() # check current working directory

In [8]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [9]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [10]:
train_data, test_data, test_RUL_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)

# 📍 Subtask 2: Sliding Window Method

Note:
* In training however, we need multiple examples to train. If we would only use the time series until today (and wait for the value of tomorrow to have a target), we would only have a single training example. Therefore, we use a trick: we replay the history.
* At each time step $t$, you treat the data as it would be today. You extract the features with everything you know until today (which is all data until and including $t$). The target for the features until time $t$ is the time value of time t + 1 (which we already know, because everything has already happened).
* The process of window-sliding is implemented in the function `roll_time_series`. Our window size will be 20 (we look at max 20 days in the past) and we disregard all windows which are shorter than 5 days.

Generating Data to train Forecasting typically involves the following steps:
1. Generate roling windows of size k (e.g. 20) from the time series data. It means that we take the last k time periods as relevant features, including the current time period. The target is then the next time period.
2. Extract features from the rolling windows. This can be as simple as taking the mean of the last k days, or more complex features like the slope of a linear regression. The dimension of the rolling window then would be (k, n_features) and the dimension of the target would be (1,).

--> The data would then be of size (n_samples, k, n_features) and the target would be of size (n_samples, 1).

## 1. Generate rolling windows

In [11]:
# define window size
MAX_K = 20
MIN_K = 5

In [12]:
%%time
# generate rolling windows
train_data_rolled = roll_time_series(train_data, column_id="UnitNumber", column_sort="Cycle", max_timeshift=MAX_K, min_timeshift=MIN_K)

In [13]:
train_data_rolled.head(5)

Findings:
* **Function Result**: New column `id` is generated, representing a **window id**. It is a tuple of the **`UnitNumber` (certain group)** and the **ending `Cycle` (ending time step)**.
* **Example**: Data with the `id` (1, 40) contains the original data of `UnitNumber` 1 of the last 20 `Cycle` steps until `Cycle` 40 (including `Cycle` 40).

In [14]:
# check if example interpretation is correct and if the window is generated correctly
rolled_win = train_data_rolled[train_data_rolled["id"] == (1, 40)]
print(rolled_win.shape)
display(rolled_win.head(5))

original_win_data = train_data[(train_data["UnitNumber"] == 1) &
                               (train_data["Cycle"] <= 40) &
                               (train_data["Cycle"] >= 40 - 20)]
print(original_win_data.shape)
display(original_win_data.head(5))

In [15]:
print(f"Number of samples: {train_data_rolled.shape[0]}")
print(f"Number of unique windows: {train_data_rolled['id'].nunique()}")
print(f"Number of windows shorter than 5 days: {train_data_rolled['id'].value_counts().loc[lambda x: x < MIN_K].sum()}")
print(f"Number of unique units: {train_data_rolled['UnitNumber'].nunique()}")

In [16]:
# check windows sizes / lengths
train_data_rolled.groupby("id").size().value_counts()

In [17]:
# get list of features
features = config["dataloading"]["features"]["operational_settings"] + \
           config["dataloading"]["features"]["sensor_measure"]

Findings:
* The fact that there are always a number of 100 windows for each window size less than 21 and more than 5 indicates that there are 100 unique units in the data and that the windows are generated correctly.

## 2. Extract Features
Note:
* We have many features and the `RUL` column is the target. so we thus need to extract features for each feature column in the window and then also get a y value for each window.

In [18]:
%%time
# extract features - tsfresh
X = extract_features(train_data_rolled.drop(["UnitNumber"], axis=1),
                     column_id="id", column_sort="Cycle",
                     default_fc_parameters=MinimalFCParameters(),
                     impute_function=impute, show_warnings=False)
# add index names
X.index = X.index.rename(["UnitNumber", "Cycle"])

In [19]:
X.head(5)

In [20]:
# check if example window can also be found in the extracted features
X.loc[(1, 40)]

## 3. Extract Target & Map to Features
Note:
* **Target Example**: The target for the row with the id (1, 40) is the RUL of the row with the id (1, 41) and so on --> shift the RUL column by one and then we can use it as our target. - Only if RUL is shifted by one for forcasting. (DEPRECATED)
* **Target Example**: The target for the row with the id (1, 40) is the RUL of the row with the id (1, 40)

In [57]:
# calculate RUL
train_data_RUL = calculate_RUL(data=train_data, time_column="Cycle", group_column="UnitNumber")

In [58]:
train_data_RUL[(train_data_RUL["UnitNumber"] == 1)].tail(5)

In [89]:
# extract target
#y = train_data_RUL.set_index(["UnitNumber", "Cycle"]).sort_index().RUL.shift(-1) # shift RUL by one for forcasting (DEPRECATED)
y = train_data_RUL.set_index(["UnitNumber", "Cycle"]).sort_index().RUL

In [85]:
# consistency check for example window (1, 40)
print(f"y value for window (1, 40): {y.loc[(1, 40)]}") # should be the same as the RUL of the row with the id (1, 41)
print(f"Train data RUL value for UnitNumber, Cycle (1, 41): {train_data_RUL[(train_data_RUL['UnitNumber'] == 1) & (train_data_RUL['Cycle'] == 41)]['RUL'].values[0]}")

Findings:
* The target value RUL does indeed match the RUL of the next cycle in the original data and the rolled data. If RUL is shifted by one for forcasting.

In [90]:
# compare index and size of X and y
print(f"Shape of X: {X.shape}")
print(f"Fist 3 rows of index of X: {X.index[:3]}")
print(f"Shape of y: {y.shape}")
print(f"Fist 3 rows of index of y: {y.index[:3]}")

In [91]:
# check last rows of y
y.tail(5)

In [92]:
# show nan values in y
print(f"Number of NaN values in y: {y.isnull().sum()}")
y[y.isnull()]

Findings:

There are some inconsistencies! :
* X is missing the first 5 dates (as our minimum window size was 5), which can be seen looking at the index length of X and y. X index has a length of `20131` and y index has a length of `20631`. This `500` difference is because we have `100` units and the first `5` cycles of each unit are not in X, thus we have $100 * 5 = 500$ missing rows in X which are in y. --> drop the first min_k time steps / cycles from X
* y is missing the last date, as we cannot predict the RUL of the last cycle of each unit, as there is no next cycle to predict on. This can be seen by looking at the last row of y where the RUL is `NaN`. - only if shifted by one for forcasting. (DEPRECATED)

In [93]:
# make X and y consistent
y = y[y.index.isin(X.index)]
X = X[X.index.isin(y.index)]

In [94]:
# compare index of X and y
print(f"Shape of X: {X.shape}")
print(f"Fist 3 rows of index of X: {X.index[:3]}")
print(f"Shape of y: {y.shape}")
print(f"Fist 3 rows of index of y: {y.index[:3]}")

In [97]:
print(f"Shape of X: {X.shape}")
display(X.tail(5))
print(f"Shape of y: {y.shape}")
display(y.tail(5))

## 3. Modification for Test Set
Note:
* The test is different from the training set, as we have extern RUL target values for the test set. This target consits only of one value for each unit, which is the RUL of the last cycle of each unit. Thus, we need to filter the data.
* Only use last window of each unit in the test set. The target RUL is in the `test_RUL_data` dataframe, which is the true RUL of the last cycle of each unit, thus a shape of (100, 1).

--> Filter rolled data to only include the last window of each UnitNumber:
* Ensure the selected cycles are sorted in ascending order of UnitNumber and Cycle values.
* Group the filtered data by UnitNumber.
* For each UnitNumber group, select the last max_window_size cycles.


In [31]:
# generate rolling windows for test data
test_data_rolled = roll_time_series(test_data, column_id="UnitNumber", column_sort="Cycle", max_timeshift=MAX_K, min_timeshift=MIN_K)

In [32]:
test_data_rolled.head(5)

In [33]:
# filter to only include the last window of each unit
filtered_test_data_rolled = test_data_rolled.groupby("UnitNumber").tail(MAX_K)

In [34]:
filtered_test_data_rolled.head(40)

In [35]:
# extract features for test data
X_test = extract_features(filtered_test_data_rolled.drop(["UnitNumber"], axis=1),
                          column_id="id", column_sort="Cycle",
                          default_fc_parameters=MinimalFCParameters(),
                          impute_function=impute, show_warnings=False)
# add index names
X_test.index = X_test.index.rename(["UnitNumber", "Cycle"])

In [36]:
# extract target for test data - match index of y_test with X_test
y_test = test_RUL_data
y_test.index = X_test.index

In [37]:
# check if X_test and y_test match
print(f"Shape of X_test: {X_test.shape}")
display(X_test.head(5))
print(f"Shape of y_test: {y_test.shape}")
display(y_test.head(5))

## 4. Summarize in Function

In [51]:
from src.data_preprocessing import create_rolling_windows_datasets

In [98]:
%%time
X_train, y_train, X_test, y_test = create_rolling_windows_datasets(train_data, test_data, test_RUL_data, column_id="UnitNumber", column_sort="Cycle", max_timeshift=20, min_timeshift=5)

In [101]:
# check if X_train and y_train match
print(f"Shape of X_train: {X_train.shape}")
display(X_train.head(5))
print(f"Shape of y_train: {y_train.shape}")
display(y_train.head(5))

In [100]:
# check if X_test and y_test match
print(f"Shape of X_test: {X_test.shape}")
display(X_test.head(5))
print(f"Shape of y_test: {y_test.shape}")
display(y_test.head(5))

In [41]:
# save processed data (as pickle)
timestamp = time.strftime("%Y%m%d-%H%M%S")
X_train.to_pickle(f"{config['paths']['processed_data_dir']}ex2_X_train_{timestamp}.pkl")
y_train.to_pickle(f"{config['paths']['processed_data_dir']}ex2_y_train_{timestamp}.pkl")
X_test.to_pickle(f"{config['paths']['processed_data_dir']}ex2_X_test_{timestamp}.pkl")
y_test.to_pickle(f"{config['paths']['processed_data_dir']}ex2_y_test_{timestamp}.pkl")

# Exkurs - Extract Features not using tsfresh